Import Necessary Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind, kruskal

# Load data
df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')

# KPI creation
df['HasClaim'] = df['TotalClaims'] > 0
df['ClaimFrequency'] = df['HasClaim'].astype(int)
df['ClaimSeverity'] = np.where(df['TotalClaims'] > 0, df['TotalClaims'], np.nan)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

df.head()

C:\Users\mihret.mekbebe\AppData\Local\Temp\ipykernel_40184\1717666586.py:7: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,HasClaim,ClaimFrequency,ClaimSeverity,Margin
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,False,0,NaN,21.929825
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,False,0,NaN,21.929825
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,False,0,NaN,0.000000
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,False,0,NaN,512.848070
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,False,0,NaN,0.000000


Null Hypothesis 1: There are no risk differences across provinces

In [ ]:
h, p = chi2_contingency(pd.crosstab(df['Province'], df['HasClaim']))[:2] # Chi-Squared test
p

np.float64(5.925510718204678e-19)

Given that the p-value is much lesser than 0.05, it shows that there is a risk difference across provinces.

Result : REJECT NULL HYPOTHESIS

Null Hypothesis 2: There are no risk differences between zip codes

In [5]:
groups = [g['ClaimFrequency'].dropna() for _, g in df.groupby('PostalCode')]
h, p = kruskal(*groups)
p

np.float64(3.1530746101963024e-30)

Similarly, given that the p-value is much lesser than 0.05 it shows that there is a risk difference across Postal codes.

Result : REJECT NULL HYPOTHESIS

Null Hypothesis 3: There is no significant margin (profit) difference between zip codes

In [15]:
groups = [g['Margin'].dropna() for _, g in df.groupby('PostalCode') if len(g) >= 5]
h, p = kruskal(*groups)
p

np.float64(0.0)

The p-value is equal to 0, therefore there are statistically significant differences in margin across Postal codes. This indicates that Postal Codes are an important variable for margins i.e. profit.

Result : REJECT NULL HYPOTHESIS

Null Hypothesis 4: There is no significant risk difference between Women and Men

In [ ]:
male = df[df['Gender']=='Male']['ClaimFrequency']
female = df[df['Gender']=='Female']['ClaimFrequency']
ttest_ind(male, female, nan_policy='omit')


TtestResult(statistic=np.float64(0.20125746666968256), pvalue=np.float64(0.8404980845002314), df=np.float64(49570.0))

The p-value is significantly high, so we fail to reject this null hypothesis. There isn't a statistically significant difference in risk based on Gender. So it shouldn't be used as an important variable for varaition in claims.

Result : FAIL TO REJECT NULL HYPOTHESIS